# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display, HTML
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()
current_brochure = ""

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [28]:
print(get_links_user_prompt(ed))

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [7]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [29]:
print(get_all_details("https://anthropic.com"))

In [21]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in html.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Include information and witty Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [9]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [38]:
def create_brochure(company_name, url):
    global current_brochure
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    current_brochure = result
    display(HTML(current_brochure))

In [25]:
print(current_brochure)

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>UrVenue Company Brochure</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            line-height: 1.6;
            margin: 20px;
            padding: 20px;
            background-color: #f2f2f2;
        }
        h1 {
            color: #333;
        }
        h2 {
            color: #555;
        }
        .section {
            margin-top: 20px;
            padding: 15px;
            background: white;
            border-radius: 8px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
    </style>
</head>
<body>
    <div class="section">
        <h1>Welcome to UrVenue</h1>
        <p>UrVenue is an innovative hospitality booking platform and venue management software designed to simplify operations and elevate guest experiences across various hospitality sectors.</p>
    </div>

    <

In [49]:
def translate_brochure(language):
    translate_system_prompt = "Translate the website into " + language
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translate_system_prompt},
            {"role": "user", "content":  current_brochure}
          ],
    )
    result = response.choices[0].message.content.replace("```html","").replace("```","")
    display(HTML(result))

In [50]:
translate_brochure("spanish")

In [24]:
create_brochure("UrVenue", "https://urvenue.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.urvenue.com'}, {'type': 'about page', 'url': 'https://www.urvenue.com/who-we-serve/nightlife-daylife/'}, {'type': 'about page', 'url': 'https://www.urvenue.com/who-we-serve/pools-and-beaches/'}, {'type': 'about page', 'url': 'https://www.urvenue.com/who-we-serve/bars-and-lounges/'}, {'type': 'about page', 'url': 'https://www.urvenue.com/who-we-serve/mixed-use-entertainment-venues/'}, {'type': 'about page', 'url': 'https://www.urvenue.com/who-we-serve/sportsbooks-sports-bars/'}, {'type': 'careers page', 'url': 'https://www.urvenue.com/request-demo/'}, {'type': 'products page', 'url': 'https://www.urvenue.com/products/venue-management-software/'}, {'type': 'products page', 'url': 'https://www.urvenue.com/products/creative-services/'}, {'type': 'products page', 'url': 'https://www.urvenue.com/products/marketplace/'}, {'type': 'news page', 'url': 'https://www.urvenue.com/news/'}, {'type': 'contact page', 'url': 'https://w

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [35]:
stream_brochure("UrVenue", "https://urvenue.com")

Found links: {'links': [{'type': 'homepage', 'url': 'https://www.urvenue.com'}, {'type': 'products page', 'url': 'https://www.urvenue.com/products/venue-management-software/'}, {'type': 'products page', 'url': 'https://www.urvenue.com/products/creative-services/'}, {'type': 'products page', 'url': 'https://www.urvenue.com/products/marketplace/'}, {'type': 'who we serve page', 'url': 'https://www.urvenue.com/who-we-serve/nightlife-daylife/'}, {'type': 'who we serve page', 'url': 'https://www.urvenue.com/who-we-serve/pools-and-beaches/'}, {'type': 'who we serve page', 'url': 'https://www.urvenue.com/who-we-serve/bars-and-lounges/'}, {'type': 'who we serve page', 'url': 'https://www.urvenue.com/who-we-serve/mixed-use-entertainment-venues/'}, {'type': 'who we serve page', 'url': 'https://www.urvenue.com/who-we-serve/sportsbooks-sports-bars/'}, {'type': 'news page', 'url': 'https://www.urvenue.com/news/'}, {'type': 'faq page', 'url': 'https://www.urvenue.com/faqs/'}, {'type': 'support page'


# 🌟 Welcome to UrVenue: Where Every Event is a Grand Adventure! 🎉

## The Best Hospitality Booking Platform & Venue Management Software!

### Who is UrVenue?
Strap on your party shoes because UrVenue is your ultimate sidekick in the wild world of nightlife and entertainment! Whether you manage a nightclub, beach resort, or a cozy bar in the heart of Las Vegas (yes, that's right - the city that never sleeps), our all-in-one platform is here to unleash the full potential of your venue. 

### Services We Offer (Because Who Doesn't Like Choices?)
- **Venue Management Software**: Unify booking, management, and guest services. Say goodbye to cluttered spreadsheets! 
- **Creative Services**: Want to look good? We’ll create seamless, branded websites giving every venue its well-deserved makeover! 
- **UV Marketplace**: Explore our curated network of third-party partners and sprinkle a little extra magic on your venue’s offerings.

### Who's Loving Us?
Our clients range from:
- **Partying Nightclubs & Lush Daylife Venues**: Let’s help you sparkle like that disco ball!
- **Sunny Pools & Beaches**: Because nothing says relaxation like sipping cocktails with your toes in the sand.
- **Mixed-Use Entertainment**: Variety is the spice of life, and we bring the whole spice rack!
- **Sportsbooks & Bars**: Not just for cheering fans – we optimize operations to generate superfan-level sales!

### 🏙️ Hello, Las Vegas! 🎲
Where the lights are brighter and the drinks are never watered down! UrVenue calls this city home, and we embrace its vibrant, spirited culture. We perfectly blend technology with the unique hospitality Las Vegas is famous for. Plus, our central location on the Strip means we’re always close to the action! 

### Why Choose UrVenue?
- **Efficiency Meets Innovation**: Manage everything from bookings to guest interactions in one centralized tool – it's like having a Swiss Army knife, but, you know, for venues.
- **Unified Booking Journey**: Our booking system is smoother than your uncle's moves at the family wedding.
- **Revenue Boosting Strategies**: We’ve got upselling strategies that’ll make your cash register sing (or maybe just jingle) with that sweet cha-ching sound!

### Join Our Fun-Tastic Team! 🎉
Do you love hospitality as much as we do? 
- We're on the lookout for innovative thinkers, party starters, and technology wizards to join our UrVenue family! 
- Flexible working conditions, an open culture, and the chance to be part of something great (with occasional office parties). 

### Ready to Join the Party?
Book a Demo, and let's transform your venue into the hottest spot in town! Because at UrVenue, we’re all about elevating experiences while keeping it fun and fabulous.

---

**UrVenue**  
Your all-in-one hospitality partner  
Las Vegas, NV  
Join the fun: [Book a Demo Today](#)



In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

---

## 📢 Welcome to Hugging Face

### **The AI Community Building the Future**

Hugging Face is a vibrant platform at the forefront of artificial intelligence and machine learning, providing a collaborative environment for developers, researchers, and organizations to create, discover, and collaborate on machine learning models, datasets, and applications.

---

## 🚀 What We Offer

### **Models, Datasets, and Spaces:**
- **400k+ Models** – Browse and utilize a vast collection of state-of-the-art machine learning models across varying functionalities.
- **100k+ Datasets** – Access and contribute to a rich library of datasets tailored for different domains including NLP, computer vision, and audio processing.
- **150k+ Applications** – Explore an extensive array of applications and experiments driven by the community.

### **Innovative Tools:**
- **Open Source Library** – Our Transformers library supports state-of-the-art ML for both PyTorch and TensorFlow.
- **Compute Solutions** – Fast, reliable deployment options with pricing starting at $0.60/hour for GPU services.
- **Enterprise Solutions** – Comprehensive support and security for businesses wanting to leverage AI technologies.

---

## 🏗️ Company Culture

At Hugging Face, we believe in building open-source technology with the community. Our culture fosters collaboration, innovation, and inclusivity, where everyone has the opportunity to contribute and shape the future of machine learning. We encourage all team members to share their work and build their portfolio while being part of a supportive community.

- **Team Collaboration** – Work together remotely and across disciplines to push the boundaries of technology.
- **Diversity and Inclusion** – An open environment that values different perspectives, backgrounds, and experiences.

---

## 🌐 Our Customers

More than **50,000 organizations** are using Hugging Face, including prominent names such as:

- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Intel**

Our platform supports various enterprises, nonprofits, and individuals who are leveraging AI technologies in diverse applications.

---

## 🙋‍♀️ Careers at Hugging Face

Join us in our mission to democratize AI! We are always looking for passionate, talented individuals eager to make an impact in the AI community. Our careers section highlights various roles in engineering, research, product management, and more.

### **Why Work With Us?**
- Impactful work in a growing technology sector.
- Opportunities for career growth and professional development.
- Flexible working conditions and a focus on work-life balance.

---

## 🌍 Join the Hugging Face Revolution!

Whether you’re a developer, an organization looking for cutting-edge machine learning solutions, or an individual wanting to be part of an AI-driven community, Hugging Face welcomes you. 

**Sign up today** and start your journey with us at [Hugging Face](https://huggingface.co). 

--- 

*For more information, follow us on our social media platforms: [GitHub](http://github.com/huggingface), [Twitter](http://twitter.com/huggingface), [LinkedIn](http://linkedin.com/company/hugging-face) and join our communities!*

In [29]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}



# Hugging Face: The AI Community Building the Future

## About Us
Hugging Face is a premier platform where the machine learning community collaborates on models, datasets, and applications. Dedicated to fostering an open-source environment, we empower over 50,000 organizations worldwide, including tech giants like Amazon, Google, and Microsoft, to innovate and advance their AI capabilities.

## Our Offerings
- **Models**: Access an extensive repository of over 400,000 machine learning models, including state-of-the-art options for text, image, and audio processing.
- **Datasets**: Explore more than 100,000 datasets tailored for various machine learning tasks, facilitating research and application building.
- **Spaces**: Showcase applications built using our technology, providing a platform for collaborative innovation.
- **Enterprise Solutions**: Our enterprise-grade offerings come with advanced security, dedicated support, and tailored resources to help businesses thrive in the AI landscape.

### Key Features
- **Open Source**: At the heart of our success lies a commitment to open-source collaboration. Our libraries, like Transformers, provide industry-leading machine learning tools across multiple frameworks. 
- **Compute Solutions**: Optimize and scale your AI workloads with our GPU-accelerated services starting from $0.60/hour.
- **Community Engagement**: Join a vibrant community of developers and researchers, contribute to our platforms, and be part of the future of AI.

## Company Culture
At Hugging Face, we believe in building a diverse and inclusive environment where creativity and collaboration thrive. Our culture centers around empowering individuals to take initiative and inspire innovation. We foster an atmosphere of curiosity and continuous learning, ideal for those passionate about technology and artificial intelligence.

## Careers
We're always on the lookout for talent! Explore opportunities to join our team in various roles across engineering, product management, and community engagement. If you're eager to push the boundaries of AI technology and collaborate with experts in the field, Hugging Face is the place for you.

### Join Us!
- [Explore Job Openings](https://huggingface.co/jobs)
- [Learn About Our Work Culture](https://huggingface.co/about)

## Our Customers
Hugging Face supports a broad customer base spanning both startups and established enterprises. Notable users include:
- **AI at Meta**: Engaging with cutting-edge research and application development.
- **Amazon Web Services**: Leveraging our models for scalable AI solutions.
- **Google**: Collaborating in advancing machine learning research.

---

**Connect with Us:**
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.gg/huggingface)

### Join the AI Revolution!
Explore our platform and be part of the movement shaping the future of artificial intelligence.



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>